In [0]:
import findspark
findspark.init('/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
import pandas as pd


spark = SparkSession.builder.appName("analytics").getOrCreate()

# K-Means Clustering


## Ingestión de Datos:

Los datos se cargan en un DataFrame.

In [0]:
# Cargamos el dataset
data_path = "data/heart_failure_clinical_records_dataset.csv"
data = spark.read.csv(data_path, header=True, inferSchema=True)

## Exploración de Datos:

* df.describe(): Proporciona un resumen estadístico de los datos.
* df.show(): Muestra las primeras filas del DataFrame para inspección visual.


In [0]:
# Mostramos las primeras filas del dataset
data.show(5)

## Selección de datos


In [0]:
# Seleccionamos las características que vamos a utilizar para el clustering
features = ['age', 
            'creatinine_phosphokinase', 
            'ejection_fraction', 
            'platelets', 
            'serum_creatinine', 
            'serum_sodium', 
            'time']


## VectorAssembler

Combina varias columnas de características en un solo vector de características, esencial para el entrenamiento del modelo.

In [0]:
# Vectorizamos las características seleccionadas
vector_assembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")
data_unscaled = vector_assembler.transform(data)

## Scaler

Utiliza `StandardScaler` para escalar las características y se entrena el modelo de escalado

### Por qué es Necesario el Escalado de Características

* Uniformidad de Escalas: Muchos algoritmos de aprendizaje automático funcionan mejor cuando las características están en la misma escala. Por ejemplo, en el caso del clustering K-Means, la distancia euclidiana se utiliza para asignar puntos a clusters. Si las características tienen escalas muy diferentes (por ejemplo, una característica está en el rango de 0 a 1 y otra en el rango de 0 a 1000), las características con rangos mayores dominarán la distancia euclidiana, y el modelo no funcionará correctamente.

* Convergencia Más Rápida: Algunos algoritmos, como el gradiente descendente, convergen más rápidamente cuando las características están escaladas. Esto se debe a que la función de pérdida tiene una forma más regular, lo que facilita la optimización.

* Mejora del Rendimiento: En algunos casos, el rendimiento del modelo puede mejorar significativamente cuando las características están escaladas. Esto es especialmente cierto para algoritmos que son sensibles a las magnitudes de las características.



In [0]:
# Escalamos las características
scaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
scaler_model = scaler.fit(data_unscaled)
data_scaled = scaler_model.transform(data_unscaled)

## Modelo K-Means

Entrena el modelo utilizando el algoritmo de Random Forest, ideal para tareas de regresión debido a su capacidad para manejar grandes conjuntos de datos y su robustez frente a datos ruidosos.

In [0]:
# Definimos el modelo KMeans
kmeans = KMeans(featuresCol="features", k=3, seed=1)

# Entrenamos el modelo
kmeans_model = kmeans.fit(data_scaled)

## Clustering

In [0]:
# Realizamos el clustering
predictions = kmeans_model.transform(data_scaled)

## Revisión de resultados del modelo

In [0]:
# Mostramos los resultados
predictions.select('age', 
                   'creatinine_phosphokinase', 
                   'ejection_fraction', 
                   'platelets', 
                   'serum_creatinine', 
                   'serum_sodium', 
                   'time', 
                   'prediction').show()

# imprimimos los centroides de los clusters
centers = kmeans_model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)